In [22]:
%pip install lxml
#%pip install rdflib
import os
import s3fs
import gzip
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd

os.environ["AWS_ACCESS_KEY_ID"] = 'PVZT77T3X1860L6FCJFG'
os.environ["AWS_SECRET_ACCESS_KEY"] = '8Uq1Hnf5y5f5K47efWXkZJ6kdQiyOy9IBa5D9MnN'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJQVlpUNzdUM1gxODYwTDZGQ0pGRyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzY5NjEyMTg2LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzcxMzE1NTA1LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc3MDcxMDcwNSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDo1ZjM1MWE3MS03NTIwLWYyNjYtNjBjOS1iYWY5NjYzMTEyYjciLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6Ijc4ODAyYmFkLTFjZTEtODFkOC1mYjI4LTQwNWY4MWQ0ZGVmNSIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.Dlw2Tbo6kDKyKT70uHQjFhqBaOTtzYXw5t9pKQu7I-GxTbbL_qo7AMAHIWUfsPj-l4JDZUf6RabEsg-Rx3A39g'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

path = "lab/dblp (1).xml.gz"
dtd_path = "lab/dblp.dtd"
rdf_path="lab/dblp.nt.gz"

with fs.open("s3://lab/conf_net_dec.parquet") as f:
    df = pd.read_parquet(f)


Note: you may need to restart the kernel to use updated packages.


In [27]:
import re
from lxml import etree
import gzip

articles = []

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        # Set up parser to load DTD and resolve entities
        # DBLP header encoding
    

        context = etree.iterparse(
            gz,
            events=("end",),
            tag="inproceedings",       # load external DTD
            resolve_entities=False,  # replace named entities like &eacute;
            recover=True
        )

        for event, elem in context:
            raw_xml = etree.tostring(elem, encoding="unicode", with_tail=False)

            # Extract content between tags manually
            def extract_tag(tag):
                m = re.search(rf"<{tag}>(.*?)</{tag}>", raw_xml, re.DOTALL)
                return m.group(1) if m else None

            title = extract_tag("title")
            conf  = extract_tag("booktitle")
            year  = extract_tag("year")
            doi   = extract_tag("ee")

            authors_raw = re.findall(r"<author>(.*?)</author>", raw_xml)
            authors = [{"name": a, "orcid": None} for a in authors_raw]

            articles.append({
                "title": title,
                "conf": conf,
                "year": year,
                "authors": authors,
                "dblp_uri": elem.get("key"),
                "doi": doi
            })

            elem.clear()
            while elem.getprevious() is not None:
                del elem.getparent()[0]

import pandas as pd
df = pd.DataFrame(articles)

In [19]:
import html

# Function to decode entities in a string or None
def decode_entities(s):
    if s is None:
        return None
    return html.unescape(s)

# Apply to your DataFrame
for col in ["title", "journal", "doi"]:
    df[col] = df[col].apply(decode_entities)

# Authors
for i, row in df.iterrows():
    for a in row["authors"]:
        a["name"] = decode_entities(a["name"])


,title,conf,year,authors,dblp_uri,doi
0,The Future of Classic Data Administration: Obj...,SWEE,1998,"[{'name': 'Arnon Rosenthal', 'orcid': None}]",www/org/mitre/future,http://www.mitre.org/support/swee/rosenthal.html
1,Some Patterns of Convincing Software Engineeri...,Denert Award,2020,"[{'name': 'Lutz Prechelt', 'orcid': None}]",books/sp/22/Prechelt22,None
2,Static Worst-Case Analyses and Their Validatio...,Denert Award,2020,"[{'name': 'Peter W&auml;gemann', 'orcid': None}]",books/sp/22/Wagemann22,None
3,Crossing Disciplinary Borders to Improve Requi...,Denert Award,2020,"[{'name': 'Anne Hess', 'orcid': None}]",books/sp/22/Hess22,None
4,What You See Is What You Get: Practical Effect...,Denert Award,2020,[],books/sp/22/Brachthauser22,None
...,...,...,...,...,...,...
3749511,"Toward a Modern Geography of Minds, Machines, ...",PT-AI,2011,"[{'name': 'Selmer Bringsjord', 'orcid': None},...",series/sapere/BringsjordG13,https://doi.org/10.1007/978-3-642-31674-6_11
3749512,Emotional Control-Conditio Sine Qua Non for Ad...,PT-AI,2011,[],series/sapere/Gros13,https://doi.org/10.1007/978-3-642-31674-6_14
3749513,Becoming Digital: Reconciling Theories of Digi...,PT-AI,2011,"[{'name': 'Harry Halpin', 'orcid': None}]",series/sapere/Halpin13,https://doi.org/10.1007/978-3-642-31674-6_15
3749514,Generative Artificial Intelligence.,PT-AI,2011,"[{'name': 'Tijn van der Zant', 'orcid': None},...",series/sapere/ZantKS13,https://doi.org/10.1007/978-3-642-31674-6_8


In [21]:
with fs.open("s3://lab/art_net_dec.parquet", "wb") as f:
            df.to_parquet(f, index=False)

In [13]:
authors_by_pid = {}
dois_by_pub = {}

signature_to_pid = {}
pub_to_signatures = {}
pub_to_pids = {}
pid_to_label = {}

RDFS_LABEL = "http://www.w3.org/2000/01/rdf-schema#label"

with fs.open(rdf_path, "rb") as f:
    with gzip.GzipFile(fileobj=f) as gz:
        for i, line in enumerate(gz):

            line = line.decode("utf-8", errors="ignore").strip()
            if not line or line.startswith("#"):
                continue

            parts = line.split(" ", 2)
            if len(parts) != 3:
                continue

            subj, pred, obj = parts
            subj = subj.strip("<>")
            pred = pred.strip("<>")
            obj = obj.strip(" .").strip("<>")

            # ---------- PID -> label ----------
            if pred == RDFS_LABEL:
                pid_to_label[subj] = obj.strip('"')

            # ---------- signature -> PID ----------
            elif pred.endswith("signatureCreator"):
                signature_to_pid[subj] = obj

            # ---------- publication -> signature ----------
            elif pred.endswith("hasSignature") or pred.endswith("authoredBy"):
                pub_to_signatures.setdefault(subj, []).append(obj)

            # ---------- DOI ----------
            elif pred.endswith("doi"):
                dois_by_pub[subj] = obj.strip('"')

            if i % 1_000_000 == 0 and i > 0:
                print(f"Processed {i:,} lines")

# ---------- FINAL MERGE ----------
for sig, pid in signature_to_pid.items():
    label = pid_to_label.get(pid)
    if label:
        authors_by_pid[pid] = label

for pub, sigs in pub_to_signatures.items():
    pids = []

    for sig in sigs:
        pid = signature_to_pid.get(sig)
        if pid:
            pids.append(pid)

    if pids:
        pub_to_pids[pub] = pids

pub_to_pids[pub] = list(set(pids))

print("Authors:", len(authors_by_pid))
print("DOIs:", len(dois_by_pub))


Processed 1,000,000 lines
Processed 2,000,000 lines
Processed 3,000,000 lines
Processed 4,000,000 lines
Processed 5,000,000 lines
Processed 6,000,000 lines
Processed 7,000,000 lines
Processed 8,000,000 lines
Processed 9,000,000 lines
Processed 10,000,000 lines
Processed 11,000,000 lines
Processed 12,000,000 lines
Processed 13,000,000 lines
Processed 14,000,000 lines
Processed 15,000,000 lines
Processed 16,000,000 lines
Processed 17,000,000 lines
Processed 18,000,000 lines
Processed 19,000,000 lines
Processed 20,000,000 lines
Processed 21,000,000 lines
Processed 22,000,000 lines
Processed 23,000,000 lines
Processed 24,000,000 lines
Processed 25,000,000 lines
Processed 26,000,000 lines
Processed 27,000,000 lines
Processed 28,000,000 lines
Processed 29,000,000 lines
Processed 30,000,000 lines
Processed 31,000,000 lines
Processed 32,000,000 lines
Processed 33,000,000 lines
Processed 34,000,000 lines
Processed 35,000,000 lines
Processed 36,000,000 lines
Processed 37,000,000 lines
Processed 

In [24]:
pd.DataFrame(pub_to_pids.items(), columns=["pid", "name"])

,pid,name
0,https://dblp.org/rec/reference/vision/Singh14,[https://dblp.org/pid/78/459-1]
1,https://dblp.org/rec/reference/vision/Wong14,[https://dblp.org/pid/69/220]
2,https://dblp.org/rec/reference/vision/Pont14b,[https://dblp.org/pid/99/2633]
3,https://dblp.org/rec/reference/vision/Alexander14,[https://dblp.org/pid/37/6152]
4,https://dblp.org/rec/reference/vision/Fukui14,[https://dblp.org/pid/01/1485]
...,...,...
8101479,https://dblp.org/rec/persons/Codd71b,[https://dblp.org/pid/c/EFCodd]
8101480,https://dblp.org/rec/persons/Codd71,[https://dblp.org/pid/c/EFCodd]
8101481,https://dblp.org/rec/persons/Hall74,[https://dblp.org/pid/82/5885]
8101482,https://dblp.org/rec/persons/Codd69,[https://dblp.org/pid/c/EFCodd]


In [ ]:
articles=[]

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        context = etree.iterparse(
            gz, events=("end",), tag="article", recover=True
        )

        for event, elem in context:
            title = elem.findtext("title")
            journal = elem.findtext("journal")
            pub_date = elem.findtext("year")
            doi = elem.findtext("ee")

            # Article-level DBLP PID
            article_pid = elem.get("key")

            # Collect authors + IDs
            authors = []

            for a in elem.findall(".//author"):
                name = a.text     # DBLP person ID
                orcid = a.get("orcid")
                if name:
                    authors.append({
                        "name": name,
                        "orcid": orcid
                    })

            articles.append({
                "title": title,
                "journal": journal,
                "year": pub_date,
                "authors": authors,
                "dblp_uri": article_pid,
                "doi": doi
            })

            elem.clear()

df = pd.DataFrame(articles)


In [2]:
articles=[]

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        context = etree.iterparse(
            gz, events=("end",), tag="inproceedings", recover=True
        )

        for event, elem in context:
            title = elem.findtext("title")
            conf = elem.findtext("booktitle")
            pub_date = elem.findtext("year")
            doi = elem.findtext("ee")

            # Article-level DBLP PID
            article_pid = elem.get("key")

            # Collect authors + IDs
            authors = []

            for a in elem.findall(".//author"):
                name = a.text     # DBLP person ID
                orcid = a.get("orcid")
                if name:
                    authors.append({
                        "name": name,
                        "orcid": orcid
                    })

            articles.append({
                "title": title,
                "conf": conf,
                "year": pub_date,
                "authors": authors,
                "dblp_uri": article_pid,
                "doi": doi
            })

            elem.clear()

df = pd.DataFrame(articles)


In [3]:
df

,title,conf,year,authors,dblp_uri,doi
0,The Future of Classic Data Administration: Obj...,SWEE,1998,"[{'name': 'Arnon Rosenthal', 'orcid': None}]",www/org/mitre/future,http://www.mitre.org/support/swee/rosenthal.html
1,Some Patterns of Convincing Software Engineeri...,Denert Award,2020,"[{'name': 'Lutz Prechelt', 'orcid': None}]",books/sp/22/Prechelt22,https://doi.org/10.1007/978-3-030-83128-8_2
2,Static Worst-Case Analyses and Their Validatio...,Denert Award,2020,"[{'name': 'Peter Wgemann', 'orcid': None}]",books/sp/22/Wagemann22,https://doi.org/10.1007/978-3-030-83128-8_11
3,Crossing Disciplinary Borders to Improve Requi...,Denert Award,2020,"[{'name': 'Anne Hess', 'orcid': None}]",books/sp/22/Hess22,https://doi.org/10.1007/978-3-030-83128-8_7
4,What You See Is What You Get: Practical Effect...,Denert Award,2020,"[{'name': 'Jonathan Immanuel Brachthuser', 'or...",books/sp/22/Brachthauser22,https://doi.org/10.1007/978-3-030-83128-8_3
...,...,...,...,...,...,...
3749511,"Toward a Modern Geography of Minds, Machines, ...",PT-AI,2011,"[{'name': 'Selmer Bringsjord', 'orcid': None},...",series/sapere/BringsjordG13,https://doi.org/10.1007/978-3-642-31674-6_11
3749512,Emotional Control-Conditio Sine Qua Non for Ad...,PT-AI,2011,"[{'name': 'Claudius Gros', 'orcid': '0000-0002...",series/sapere/Gros13,https://doi.org/10.1007/978-3-642-31674-6_14
3749513,Becoming Digital: Reconciling Theories of Digi...,PT-AI,2011,"[{'name': 'Harry Halpin', 'orcid': None}]",series/sapere/Halpin13,https://doi.org/10.1007/978-3-642-31674-6_15
3749514,Generative Artificial Intelligence.,PT-AI,2011,"[{'name': 'Tijn van der Zant', 'orcid': None},...",series/sapere/ZantKS13,https://doi.org/10.1007/978-3-642-31674-6_8


In [2]:
from lxml import etree
import gzip

direct_set = set()

with fs.open(path, "rb") as f:
    with gzip.GzipFile(fileobj=f) as gz:
        context = etree.iterparse(gz, events=("end",), tag="inproceedings", recover=True)
        for event, elem in context:
            # collect child tags
            for child in elem:
                if child.tag is not None:
                    direct_set.add(child.tag)

            # clear memory: remove element and its previous siblings
            elem.clear()
            while elem.getprevious() is not None:
                del elem.getparent()[0]

print(sorted(direct_set))

['author', 'booktitle', 'cdrom', 'cite', 'crossref', 'editor', 'ee', 'month', 'note', 'number', 'pages', 'publnr', 'stream', 'title', 'url', 'volume', 'year']
